# Optional for Running on Local Machine
Run the command below to install the packages according to the configuration file requirements.txt.

In [ ]:
pip install -r requirements.txt

# About the data
The data for this project can be found [here](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). This dataset contains the real bank transactions made by European cardholders in the year 2013. As a security concern, the actual variables are not being shared but — they have been transformed versions of PCA. As a result, we can find 29 feature columns and 1 final class column.

![Data Snapshot](./images/Data%20Snapshot.webp)  

# Importing Necessary Libraries
It is a good practice to import all the necessary libraries in one place — so that we can modify them quickly.

For this credit card data, the features that we have in the dataset are the transformed version of PCA, so we will not need to perform the feature selection again. Otherwise, it is recommended to use RFE, RFECV, SelectKBest and VIF score to find the best features for your model.

In [ ]:
#Packages related to general operating system & warnings
import os 
import warnings
warnings.filterwarnings('ignore')
#Packages related to data importing, manipulation, exploratory data #analysis, data understanding
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from termcolor import colored as cl # text customization
#Packages related to data visualizaiton
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Setting plot sizes and type of plot
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Importing Dataset
Importing the dataset is pretty much simple. You can use pandas module in python to import it.

Run the below command to import your data.

In [ ]:
data=pd.read_csv("creditcard.csv")

# Data Processing & Understanding
The one main thing you will notice about this data is that — the dataset is imbalanced towards a feature. Which seems pretty valid for such kind of data. Because today many banks have adopted different security mechanisms — so it is harder for hackers to make such moves.

Still, sometimes when there is some vulnerability in the system — the chance of such activities can increase.

That’s why we can see the majority of transactions belongs to our datasets are normal and only a few percentages of transactions are fraudulent.

Let’s check the transaction distribution.

In [ ]:
Total_transactions = len(data)
normal = len(data[data.Class == 0])
fraudulent = len(data[data.Class == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print(cl('Total number of Trnsactions are {}'.format(Total_transactions), attrs = ['bold']))
print(cl('Number of Normal Transactions are {}'.format(normal), attrs = ['bold']))
print(cl('Number of fraudulent Transactions are {}'.format(fraudulent), attrs = ['bold']))
print(cl('Percentage of fraud Transactions is {}'.format(fraud_percentage), attrs = ['bold']))

Only 0.17% of transactions are fraudulent.

We can also check for null values using the following line of code.

In [ ]:
data.info()

As per the count per column, we have no null values. Also, feature selection is not the case for this use case. Anyway, you can try applying feature selection mechanisms to check if the results are optimised.

I have observed in our data 28 features are transformed versions of PCA but the Amount is the original one. And, while checking the minimum and maximum is in the amount — I found the difference is huge that can deviate our result.

In [ ]:
min(data.Amount), max(data.Amount)
#(0.0, 25691.16)

In this case, it is a good practice to scale this variable. We can use a standard scaler to make it fix.

In [ ]:
sc = StandardScaler()
amount = data['Amount'].values
data['Amount'] = sc.fit_transform(amount.reshape(-1, 1)) # type: ignore

We have one more variable which is the time which can be an external deciding factor — but in our modelling process, we can drop it.

In [ ]:
data.drop(['Time'], axis=1, inplace=True)

We can also check for any duplicate transactions. Before removing any duplicate transaction, we are having 284807 transactions in our data. Let’s remove the duplicate and observe the changes.

In [ ]:
data.shape
#(284807, 30)

Run the below line of code to remove any duplicates.

In [ ]:
data.drop_duplicates(inplace=True)

Let’s now check the count again.

In [ ]:
data.shape
#(275663, 30)

So, we were having around ~9000 duplicate transactions.

Here we go!! We now have properly scaled data with no duplicate, no missing. Let’s now split it for our model building.

# Train & Test Split
Before splitting train & test — we need to define dependent and independent variables. The dependent variable is also known as X and the independent variable is known as **y**.

In [ ]:
X = data.drop('Class', axis = 1).values
y = data['Class'].values

Now, let split our train and test data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

That’s it. We now have two different data set — Train data we will be used for training our model and the data which is unseen will be used for testing.

# Model Building
We will be trying different machine learning models one by one. Defining models are much easier. A single line of code can define our model. And, in the same way, a single line of code can fit the model on our data.

We can also tune these models by selecting different optimized parameters. But, if the accuracy is better even with less parameter tuning then — no need to make it complex.

## *Decision Tree*

In [ ]:
DT = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
DT.fit(X_train, y_train)
dt_yhat = DT.predict(X_test)

Let’s check the accuracy of our decision tree model.

In [ ]:
print('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, dt_yhat)))
#Accuracy score of the Decision Tree model is 0.9991583957281328

Checking F1-Score for the decision tree model.

In [ ]:
print('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, dt_yhat)))
#F1 score of the Decision Tree model is 0.7542372881355933

Checking the confusion matrix:

In [ ]:
confusion_matrix(y_test, dt_yhat, labels = [0, 1])
#array([[68769,    19],
#       [   39,    89]], dtype=int64)

Here, the first row represents positive and the second row represents negative. So, we have 68769 as true positive and 19 are false positive. That says, out of 68769+19=68788, we have 68769 that are successfully classified as a normal transaction and 19 were falsely classified as normal — but they were fraudulent.

Let’s now try different models and check their performance.

## *K-Nearest Neighbors*

In [ ]:
n = 7
KNN = KNeighborsClassifier(n_neighbors = n)
KNN.fit(X_train, y_train)
knn_yhat = KNN.predict(X_test)

Let’s check the accuracy of our K-Nearest Neighbors model.

In [ ]:
print('Accuracy score of the K-Nearest Neighbors model is {}'.format(accuracy_score(y_test, knn_yhat)))
#Accuracy score of the K-Nearest Neighbors model is 0.999288989494457

Checking F1-Score for the K-Nearest Neighbors model.

In [ ]:
print('F1 score of the K-Nearest Neighbors model is {}'.format(f1_score(y_test, knn_yhat)))
#F1 score of the K-Nearest Neighbors model is 0.7949790794979079

## *Logistic Regression*

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_yhat = lr.predict(X_test)

Let’s check the accuracy of our Logistic Regression model.

In [ ]:
print('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)))
#Accuracy score of the Logistic Regression model is 0.9989552498694062

Checking F1-Score for the Logistic Regression model.

In [ ]:
print('F1 score of the Logistic Regression model is {}'.format(f1_score(y_test, lr_yhat)))
#F1 score of the Logistic Regression model is 0.6666666666666666

## *Support Vector Machines*

In [ ]:
svm = SVC()
svm.fit(X_train, y_train)
svm_yhat = svm.predict(X_test)

Let’s check the accuracy of our Support Vector Machines model.

In [ ]:
print('Accuracy score of the Support Vector Machines model is {}'.format(accuracy_score(y_test, svm_yhat)))
#Accuracy score of the Support Vector Machines model is 0.999318010331418

Checking F1-Score for the Support Vector Machines model.

In [ ]:
print('F1 score of the Support Vector Machines model is {}'.format(f1_score(y_test, svm_yhat)))
#F1 score of the Support Vector Machines model is 0.7813953488372093

## *Random Forest*

In [ ]:
rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)

Let’s check the accuracy of our Random Forest model.

In [ ]:
print('Accuracy score of the Random Forest model is {}'.format(accuracy_score(y_test, rf_yhat)))
#Accuracy score of the Random Forest model is 0.9991729061466132

Checking F1-Score for the Random Forest model.

In [ ]:
print('F1 score of the Random Forest model is {}'.format(f1_score(y_test, rf_yhat)))
#F1 score of the Random Forest model is 0.7397260273972602

## *XGBoost*

In [ ]:
xgb = XGBClassifier(max_depth = 4)
xgb.fit(X_train, y_train)
xgb_yhat = xgb.predict(X_test)

Let’s check the accuracy of our XGBoost model.

In [ ]:
print('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)))
#Accuracy score of the XGBoost model is 0.999506645771664

Checking F1-Score for the XGBoost model.

In [ ]:
print('F1 score of the XGBoost model is {}'.format(f1_score(y_test, xgb_yhat)))
#F1 score of the XGBoost model is 0.8495575221238937

# Conclusion

Well, congratulation!! We just received 99.95% accuracy in our credit card fraud detection. This number should not be surprising as our data was balanced towards one class. The good thing that we have noticed from the confusion matrix is that — our model is not overfitted.

Finally, based on our accuracy score — XGBoost is the winner for our case. The only catch here is the data that we have received for model training. The data features are the transformed version of PCA. If the actual features follow a similar pattern then we are doing great!!